In [75]:
import pandas as pd
import biogeme.logging as blog
import biogeme.biogeme as bio
from biogeme import models
from biogeme.results import calculate_correlation
from biogeme.expressions import Beta

import numpy as np
import biogeme.database as db
import biogeme.optimization as opt
from biogeme.expressions import Beta, DefineVariable
import biogeme.expressions as exp


intermodal_data = pd.read_csv('/Users/lei/Documents/毕业论文/jiadingsurvey/intermodalAnalysis/2.fromRPtoSP/fitmodal_data/allmodal_data_travelinfo_social_wide.csv')

intermodal_data = intermodal_data[intermodal_data['totaldistance_car'] < 50000]
intermodal_data['totalcost_car_new'] = (intermodal_data['totaldistance_car']/1000) * 1.09
database = db.Database("intermodal_data", intermodal_data)
globals().update(database.variables)




In [65]:
# # totalcost_car_new =database.DefineVariable('totalcost_car_new', totaldistance_car/1.09)
# totalcost_metro_car_new=database.DefineVariable('totalcost_metro_car_new_5', totalcost_metro_car-totalcost_metro_car+5)

In [76]:
database.data

,index,totalduration_bus_metro,totalduration_metro,totalduration_bike_metro,totalduration_car,totalduration_ebike_metro,totalduration_metro_car,totalduration_bus,totaldistance_bus_metro,totaldistance_metro,...,departure,purpose,choice,bus_metro_av,metro_car_av,bike_metro_av,ebike_metro_av,metro_av,bus_av,totalcost_car_new
0,22,111,121,98,40,94,81,167,47696,45226,...,1,1,7,1,1,1,0,1,1,28.95476
1,47,110,131,93,35,76,76,130,34362,32086,...,1,0,6,1,1,1,0,1,1,23.35652
2,63,109,102,85,34,85,67,129,35523,43068,...,0,1,7,1,1,1,0,1,1,36.67414
3,67,104,124,108,30,108,32,107,37610,58550,...,1,1,7,1,1,1,1,1,1,18.49403
4,81,94,93,82,34,82,47,165,33442,42466,...,1,1,7,1,1,1,1,1,1,36.73409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,43138,56,57,36,24,36,25,57,9695,8658,...,1,0,7,1,1,1,0,1,1,8.42570
1821,43139,61,51,36,21,36,33,60,8411,8952,...,1,1,7,1,1,1,0,1,1,9.30206
1822,43141,101,91,74,36,74,71,173,29407,29598,...,0,1,5,1,1,1,1,1,1,36.07682
1823,43143,52,68,53,13,53,47,36,10915,17168,...,1,1,6,1,1,1,1,1,1,4.68918


In [77]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数
# B_COST_metro_car = exp.Beta('B_COST_metro_car',0,None ,None ,0)  # 出行费用前面的系数


B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS= exp.Beta('B_TRNUMS_metro',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions




V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}

MU_metro = Beta('MU_metro', 1, 1, None, 0)
MU_bus = Beta('MU_bus', 1, 1, None, 1)

metro_nest = MU_metro, [1, 2, 3, 4, 5]
bus_nest = MU_bus, [6]
car_nest = 1.0, [7]


nests = car_nest,bus_nest,metro_nest



In [78]:
logprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "nested_logit_metro_bus_car_carcost_new"

biogeme.generateHtml = True
biogeme.generatePickle = False

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_metro_bus_car_carcost_new~05.html
Pickle file:  None


In [79]:
gs = results.getGeneralStatistics()

for k,v in gs.items():
    print("{}= {}".format(k.ljust(45),v[0]))

Number of estimated parameters               = 20
Sample size                                  = 1718
Excluded observations                        = 0
Init log likelihood                          = -21976.76197802968
Final log likelihood                         = -1801.935794195769
Likelihood ratio test for the init. model    = 40349.652367667826
Rho-square for the init. model               = 0.918007220718085
Rho-square-bar for the init. model           = 0.9170971685447943
Akaike Information Criterion                 = 3643.871588391538
Bayesian Information Criterion               = 3752.849910442422
Final gradient norm                          = 0.13616915020207046
Nbr of threads                               = 10


# nested 2

In [80]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数



B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS_metro = exp.Beta('B_TRNUMS_metro',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions


V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS_metro * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS_metro * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS_metro * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS_metro * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS_metro * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS_metro * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_ACTIME_car * ac_eg_transfer_duration_car + \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}

MU_metro_bike_ebike = Beta('MU_metro_bike_ebike', 1, 1, None, 0)
MU_metro_walk = Beta('MU_metro_walk', 1, 1, None, 0)

metro_bike_ebike_nest = MU_metro_bike_ebike, [3, 4]
metro_walk_nest = MU_metro_walk, [5,6]
bus_metro_nest = 1.0 , [1]
metro_car_nest = 1.0, [2]
car_nest = 1.0, [7]



nests = car_nest,metro_car_nest,bus_metro_nest,metro_walk_nest,metro_bike_ebike_nest



In [81]:
logprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "nested_logit_metro_walk_ebikebike_car"

biogeme.generateHtml = True
biogeme.generatePickle = True

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_metro_walk_ebikebike_car~01.html
Pickle file:  nested_logit_metro_walk_ebikebike_car~00.pickle


# nested 3

非机动化，和机动化

In [82]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数



B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS_metro = exp.Beta('B_TRNUMS_metro',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions


V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS_metro * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS_metro * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS_metro * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS_metro * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS_metro * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS_metro * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_ACTIME_car * ac_eg_transfer_duration_car + \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}

MU_metro_Novehicle = Beta('MU_metro_Novehicle', 1, 1, None, 0)

MU_metro_Novehicle_nest = MU_metro_Novehicle, [3, 4,5,6]
bus_metro_nest = 1.0 , [1]
metro_car_nest = 1.0, [2]
car_nest = 1.0, [7]



nests = car_nest,metro_car_nest,bus_metro_nest,MU_metro_Novehicle_nest

logprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "nested_logit_metro_nocar_car"

biogeme.generateHtml = True
biogeme.generatePickle = True

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")



Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_metro_nocar_car.html
Pickle file:  nested_logit_metro_nocar_car.pickle


# nested 4

In [84]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数



B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS_metro = exp.Beta('B_TRNUMS_metro',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions


V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS_metro * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS_metro * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS_metro * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS_metro * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS_metro * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS_metro * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_ACTIME_car * ac_eg_transfer_duration_car + \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}

MU_metro_Novehicle = Beta('MU_metro_Novehicle', 1, 1, None, 0)

MU_metro_Novehicle_nest = MU_metro_Novehicle, [1,3, 4,5,6]
metro_car_nest = 1.0, [2]
car_nest = 1.0, [7]



nests = car_nest,metro_car_nest,MU_metro_Novehicle_nest

logprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "nested_logit_car_metro_car_transit"

biogeme.generateHtml = True
biogeme.generatePickle = True

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")



Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_car_metro_car_transit.html
Pickle file:  nested_logit_car_metro_car_transit.pickle


# nested 5

In [85]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数



B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS_metro = exp.Beta('B_TRNUMS_metro',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions


V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS_metro * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS_metro * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS_metro * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS_metro * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS_metro * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS_metro * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_ACTIME_car * ac_eg_transfer_duration_car + \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}

MU_transit_walk = Beta('MU_transit_walk', 1, 1, None, 0)
MU_metro_bike_ebike = Beta('MU_metro_bike_ebike', 1, 1, None, 0)


MU_transit_walk_nest = MU_transit_walk, [1,5,6]
MU_metro_bike_ebike_nest = MU_metro_bike_ebike, [3, 4]
metro_car_nest = 1.0, [2]
car_nest = 1.0, [7]



nests = car_nest,metro_car_nest,MU_transit_walk_nest,MU_metro_bike_ebike_nest

logprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

biogeme  = bio.BIOGEME(database, logprob)
biogeme.modelName = "nested_logit_car_metro_car_walk_bikeebike"

biogeme.generateHtml = True
biogeme.generatePickle = True

results = biogeme.estimate()

print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")



Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_car_metro_car_walk_bikeebike.html
Pickle file:  nested_logit_car_metro_car_walk_bikeebike.pickle


In [ ]:
prob_bus_metro = models.nested(V, None , nests , 1)
prob_metro_car = models.nested(V, None , nests , 2)
prob_bike_metro = models.nested(V, None , nests , 3)
prob_ebike_metro = models.nested(V, None , nests , 4)
prob_metro = models.nested(V, None , nests , 5)
prob_bus = models.nested(V, None , nests , 6)
prob_car = models.nested(V, None , nests , 7)

In [21]:
import biogeme.results as res
simulate ={'Prob. bus_metro':  prob_bus_metro ,
              'Prob. metro_car': prob_metro_car ,
                'Prob. bike_metro': prob_bike_metro ,
                'Prob. ebike_metro': prob_ebike_metro ,
                'Prob. metro': prob_metro ,
                'Prob. bus': prob_bus ,
                'Prob. car': prob_car}

biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "nested_logit_metro_walk_ebikebike_car"
betas = biogeme.free_beta_names()

print('Extracting the following variables:')
for k in betas:
    print('\t',k)

results = res.bioResults(pickleFile='nested_logit_metro_walk_ebikebike_car.pickle')
betaValues = results.getBetaValues ()

The syntax "freeBetaNames" is deprecated and is replaced by the syntax "free_beta_names".


Extracting the following variables:
	 ASC_bike_metro
	 ASC_bus
	 ASC_bus_metro
	 ASC_ebike_metro
	 ASC_metro
	 ASC_metro_car
	 B_ACTIME_bike_ebike
	 B_ACTIME_car
	 B_ACTIME_walk
	 B_COST_Transit
	 B_COST_car
	 B_TIME_bike_metro
	 B_TIME_bus
	 B_TIME_bus_metro
	 B_TIME_car
	 B_TIME_ebike_metro
	 B_TIME_metro
	 B_TIME_metro_car
	 B_TRNUMS_metro
	 MU_metro_bike_ebike
	 MU_metro_walk


In [22]:
simulatedValues = biogeme.simulate(betaValues)

In [24]:
b = results.getBetasForSensitivityAnalysis(betas , size=100,useBootstrap=False)

left, right = biogeme.confidenceIntervals(b, .9)

In [26]:
simulatedValues.columns

Index(['Prob. bus_metro', 'Prob. metro_car', 'Prob. bike_metro',
       'Prob. ebike_metro', 'Prob. metro', 'Prob. bus', 'Prob. car'],
      dtype='object')

In [28]:
marketShare_bus_metro = simulatedValues['Prob. bus_metro'].mean()
marketShare_bus_metro_left = left['Prob. bus_metro'].mean()
marketShare_bus_metro_right = right['Prob. bus_metro'].mean()

marketShare_metro_car = simulatedValues['Prob. metro_car'].mean()
marketShare_metro_car_left = left['Prob. metro_car'].mean()
marketShare_metro_car_right = right['Prob. metro_car'].mean()

marketShare_bike_metro = simulatedValues['Prob. bike_metro'].mean()
marketShare_bike_metro_left = left['Prob. bike_metro'].mean()
marketShare_bike_metro_right = right['Prob. bike_metro'].mean()

marketShare_ebike_metro = simulatedValues['Prob. ebike_metro'].mean()
marketShare_ebike_metro_left = left['Prob. ebike_metro'].mean()
marketShare_ebike_metro_right = right['Prob. ebike_metro'].mean()

marketShare_metro = simulatedValues['Prob. metro'].mean()
marketShare_metro_left = left['Prob. metro'].mean()
marketShare_metro_right = right['Prob. metro'].mean()

marketShare_bus = simulatedValues['Prob. bus'].mean()
marketShare_bus_left = left['Prob. bus'].mean()
marketShare_bus_right = right['Prob. bus'].mean()

marketShare_car = simulatedValues['Prob. car'].mean()
marketShare_car_left = left['Prob. car'].mean()
marketShare_car_right = right['Prob. car'].mean()

# print market shares unit in percentage

print(f"market share bus_metro: {100*marketShare_bus_metro:.3f} [{100*marketShare_bus_metro_left:.3f}, {100*marketShare_bus_metro_right:.3f}]")
print(f"market share metro_car: {100*marketShare_metro_car:.3f} [{100*marketShare_metro_car_left:.3f}, {100*marketShare_metro_car_right:.3f}]")
print(f"market share bike_metro: {100*marketShare_bike_metro:.3f} [{100*marketShare_bike_metro_left:.3f}, {100*marketShare_bike_metro_right:.3f}]")
print(f"market share ebike_metro: {100*marketShare_ebike_metro:.3f} [{100*marketShare_ebike_metro_left:.3f}, {100*marketShare_ebike_metro_right:.3f}]")
print(f"market share metro: {100*marketShare_metro:.3f} [{100*marketShare_metro_left:.3f}, {100*marketShare_metro_right:.3f}]")
print(f"market share bus: {100*marketShare_bus:.3f} [{100*marketShare_bus_left:.3f}, {100*marketShare_bus_right:.3f}]")
print(f"market share car: {100*marketShare_car:.3f} [{100*marketShare_car_left:.3f}, {100*marketShare_car_right:.3f}]")


    


market share bus_metro: 11.397 [9.171, 13.683]
market share metro_car: 1.808 [1.061, 3.164]
market share bike_metro: 3.420 [2.171, 4.958]
market share ebike_metro: 3.100 [2.070, 4.670]
market share metro: 19.945 [17.027, 23.283]
market share bus: 6.740 [4.958, 9.325]
market share car: 53.589 [48.467, 57.564]


In [31]:
intermodal_data['choice'].value_counts(normalize=True)

choice
7    0.535890
5    0.199452
1    0.113973
6    0.067397
3    0.036164
4    0.029041
2    0.018082
Name: proportion, dtype: float64

In [45]:
intermodal_data

,index,totalduration_bus_metro,totalduration_metro,totalduration_bike_metro,totalduration_car,totalduration_ebike_metro,totalduration_metro_car,totalduration_bus,totaldistance_bus_metro,totaldistance_metro,...,car_engine_type,departure,purpose,choice,bus_metro_av,metro_car_av,bike_metro_av,ebike_metro_av,metro_av,bus_av
0,22,111,121,98,40,94,81,167,47696,45226,...,0,1,1,7,1,1,1,0,1,1
1,47,110,131,93,35,76,76,130,34362,32086,...,0,1,0,6,1,1,1,0,1,1
2,63,109,102,85,34,85,67,129,35523,43068,...,1,0,1,7,1,1,1,0,1,1
3,67,104,124,108,30,108,32,107,37610,58550,...,0,1,1,7,1,1,1,1,1,1
4,81,94,93,82,34,82,47,165,33442,42466,...,1,1,1,7,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,43138,56,57,36,24,36,25,57,9695,8658,...,1,1,0,7,1,1,1,0,1,1
1821,43139,61,51,36,21,36,33,60,8411,8952,...,1,1,1,7,1,1,1,0,1,1
1822,43141,101,91,74,36,74,71,173,29407,29598,...,0,0,1,5,1,1,1,1,1,1
1823,43143,52,68,53,13,53,47,36,10915,17168,...,0,1,1,6,1,1,1,1,1,1


In [40]:
temp = intermodal_data['totaldistance_car']/intermodal_data['totalcost_car']

In [44]:
temp.to_clipboard()

In [ ]:
(0.4*1.09+0.6*2)